In [ ]:
# project
import os
import sys 
project_dir = os.path.join(os.path.expanduser('~'), 'git', 'diffsim-sysid')
sys.path.insert(1, project_dir)
# physics
import warp as wp
# graphics
import mitsuba as mi
# plots
from matplotlib import pyplot as plt
from src.mpl_utils import set_fig_size, set_mpl_format
# config
from src.config import cfg
# dataset
from src.sim import Simulator
from src.dataset import load_train_data

# setup
set_mpl_format()
wp.init()
mi.set_variant('llvm_ad_rgb')
print(f'Mitsuba 3 Variant: {mi.variant()}')

### Config

In [ ]:
# fill in your directory set up here
config_fp = os.path.join(project_dir, f'cfg/templates/comp-movi_a.yaml')
cfg.merge_from_file(config_fp)
cfg.path = project_dir
cfg.DATA.path = os.path.join(project_dir, 'data/sets/')
print(cfg)

### Data

In [ ]:
train_ds, ds_info = load_train_data(cfg)
example = next(iter(train_ds))
time_step = 20
vid_id = example['metadata']['video_name']
print(f'Video: {vid_id}')

### Rendering

In [ ]:
sim = Simulator(cfg, example)
recon_dict = sim.get_render_orig_timestep_dict(time_step)
recon_img = mi.render(mi.load_dict(recon_dict), spp=64)

our_dict = sim.get_render_timestep_dict(-1)
our_img = mi.render(mi.load_dict(our_dict), spp=64)

In [ ]:
fig_size = set_fig_size(subplots=(1, 3))
fig, axs = plt.subplots(1, 3, figsize=fig_size, constrained_layout=True)


movi_img = example['video'][time_step]
axs[0].imshow(movi_img)
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[0].set_title('MOVi A Snapshot')

scene_bitmap = mi.util.convert_to_bitmap(recon_img)
axs[1].imshow(scene_bitmap)
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].set_title('Mitsuba 3 Reconstruction')

scene_bitmap = mi.util.convert_to_bitmap(our_img)
axs[2].imshow(scene_bitmap)
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[2].set_title('Our Snapshot')

plt.show()
#plt.savefig('fig_movi_a_mitsuba.pdf', bbox_inches='tight')

In [ ]:
# # for backprop
# def run_phys(model, integrator, states):
#     # computation graph
#     tape = wp.Tape()
#     with tape:
#         for i in range(SIM_STEPS):
#             states[i].clear_forces()
#             wp.sim.collide(model, states[i])
#             integrator.simulate(model, states[i], states[i + 1], SIM_DT)

#     return tape